# Librairies

In [1]:
pip install pandas pyarrow

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install imbalanced-learn


Note: you may need to restart the kernel to use updated packages.


In [47]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn import set_config
from sklearn import tree
from sklearn.metrics import f1_score, recall_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import joblib

import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline  # Utiliser le Pipeline d'imblearn
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin


# Data 

In [78]:
data = pd.read_parquet('data_cleaned.parquet')
data

,ARR_DELAY_NEW,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,ORIGIN_AIRPORT_ID,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_AIRPORT_ID,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,CRS_ARR_TIME,DISTANCE
0,0.0,2016,1,1,6,3,AA,11298.0,"Dallas/Fort Worth, TX",TX,11433.0,"Detroit, MI",MI,1100.0,1438.0,986.0
1,0.0,2016,1,1,7,4,AA,11298.0,"Dallas/Fort Worth, TX",TX,11433.0,"Detroit, MI",MI,1100.0,1438.0,986.0
2,7.0,2016,1,1,8,5,AA,11298.0,"Dallas/Fort Worth, TX",TX,11433.0,"Detroit, MI",MI,1100.0,1438.0,986.0
3,0.0,2016,1,1,9,6,AA,11298.0,"Dallas/Fort Worth, TX",TX,11433.0,"Detroit, MI",MI,1100.0,1438.0,986.0
4,113.0,2016,1,1,10,7,AA,11298.0,"Dallas/Fort Worth, TX",TX,11433.0,"Detroit, MI",MI,1100.0,1438.0,986.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5556190,0.0,2016,4,12,31,6,WN,15376.0,"Tucson, AZ",AZ,12892.0,"Los Angeles, CA",CA,755.0,830.0,451.0
5556191,9.0,2016,4,12,31,6,WN,15376.0,"Tucson, AZ",AZ,12892.0,"Los Angeles, CA",CA,1320.0,1355.0,451.0
5556192,0.0,2016,4,12,31,6,WN,15376.0,"Tucson, AZ",AZ,13232.0,"Chicago, IL",IL,705.0,1125.0,1440.0
5556193,0.0,2016,4,12,31,6,WN,15376.0,"Tucson, AZ",AZ,14679.0,"San Diego, CA",CA,1220.0,1235.0,368.0


# Pipeline

In [60]:
data['ARR_DELAY_NEW'] = data['ARR_DELAY_NEW'].apply(lambda x: 1 if x != 0 else 0)

X = data[['DAY_OF_MONTH', 'MONTH', 'YEAR', 'ORIGIN_CITY_NAME', 'DEST_CITY_NAME', 'UNIQUE_CARRIER']]
y = data['ARR_DELAY_NEW']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Dummy

In [61]:
dummy_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DummyClassifier(strategy='most_frequent'))
])

In [62]:
dummy_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['DAY_OF_MONTH', 'MONTH', 'YEAR'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['ORIGIN_CITY_NAME', 'DEST_CITY_NAME', 'UNIQUE_CARRIER'], dtype='object'))])),
                ('classifier', DummyClassifier(strategy='most_frequent'))])

In [63]:
set_config(display='diagram')
dummy_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['DAY_OF_MONTH', 'MONTH', 'YEAR'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['ORIGIN_CITY_NAME', 'DEST_CITY_NAME', 'UNIQUE_CARRIER'], dtype='object'))])),
                ('classifier', DummyClassifier(strategy='most_frequent'))])

In [64]:
y_pred = dummy_pipeline.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

cv_scores = cross_val_score(dummy_pipeline, X, y, cv=5)
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())

/home/mathieu/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mathieu/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mathieu/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

Classification Report:
               precision    recall  f1-score   support

           0       0.66      1.00      0.80    733551
           1       0.00      0.00      0.00    377688

    accuracy                           0.66   1111239
   macro avg       0.33      0.50      0.40   1111239
weighted avg       0.44      0.66      0.52   1111239

Confusion Matrix:
 [[733551      0]
 [377688      0]]
Accuracy: 0.6601199201971854
Cross-validation scores: [0.65974736 0.65974736 0.65974736 0.65974646 0.65974646]
Mean cross-validation score: 0.6597470031199408


# Logistic regression

In [72]:
from sklearn.linear_model import LogisticRegression


In [73]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('undersampler', RandomUnderSampler(random_state=42)),
    ('classifier', LogisticRegression(max_iter=1000, random_state=40))
])

In [74]:
pipeline.fit(X_train, y_train)
set_config(display='diagram')
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['DAY_OF_MONTH', 'MONTH', 'YEAR'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['ORIGIN_CITY_NAME', 'DEST_CITY_NAME', 'UNIQUE_CARRIER'], dtype='object'))])),
                ('undersampler', RandomUnderSampler(random_state=42)),
                ('classifier',
                 LogisticRegression(max_iter=1000, random_state=40))])

In [75]:
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.55
              precision    recall  f1-score   support

           0       0.72      0.53      0.61    733551
           1       0.40      0.60      0.48    377688

    accuracy                           0.55   1111239
   macro avg       0.56      0.56      0.54   1111239
weighted avg       0.61      0.55      0.56   1111239

Confusion Matrix:
 [[388018 345533]
 [151620 226068]]


# Random forest

In [65]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('undersampler', RandomUnderSampler(random_state=42)),
    ('classifier', RandomForestClassifier(n_estimators=150, max_depth=8, random_state=40))
])

In [66]:
pipeline.fit(X_train, y_train)
set_config(display='diagram')
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['DAY_OF_MONTH', 'MONTH', 'YEAR'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['ORIGIN_CITY_NAME', 'DEST_CITY_NAME', 'UNIQUE_CARRIER'], dtype='object'))])),
                ('undersampler', RandomUnderSampler(random_state=42)),
                ('classifier',
                 RandomForestClassifier(max_depth=8, n_estimators=150,
                                        random_state=40))])

In [67]:
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.54
              precision    recall  f1-score   support

           0       0.72      0.49      0.58    733551
           1       0.39      0.64      0.48    377688

    accuracy                           0.54   1111239
   macro avg       0.56      0.56      0.53   1111239
weighted avg       0.61      0.54      0.55   1111239

Confusion Matrix:
 [[355952 377599]
 [137381 240307]]


# Joblib

In [76]:
joblib.dump(pipeline, 'rf_model.joblib')

['rf_model.joblib']